# adulib.llm

In [ ]:
#|default_exp llm

In [ ]:
#|hide
import nblite; from nbdev.showdoc import show_doc; nblite.nbl_export()

In [ ]:
#|export
try:
    import os
    import json
    import tempfile
    from pathlib import Path
    from openai import OpenAI, AsyncOpenAI
    import tiktoken
    from asynciolimiter import Limiter
    import diskcache
except ImportError as e:
    raise ImportError(f"Install adulib[{__name__.split('.')[-1]}] to use this API.") from e

In [ ]:
import adulib.llm

In [ ]:
#|export
model_context_windows = {
    'gpt-4o': 128000 * 0.8,
    'gpt-4o-mini': 128000 * 0.8,
}

model_rate_limits = {
    'gpt-4o': 10000,
    'gpt-4o-mini': 30000,
}

In [ ]:
show_doc(adulib.llm.tokenize_text)

---

### tokenize_text

>      tokenize_text (text, llm_model)

In [ ]:
#|export
__model_tokenisers = {}

def tokenize_text(text, llm_model):
    if llm_model not in __model_tokenisers:
        __model_tokenisers[llm_model] = tiktoken.encoding_for_model(llm_model)
    return __model_tokenisers[llm_model].encode(text)

In [ ]:
tokenize_text("hello world", 'gpt-4o')

[24912, 2375]

In [ ]:
show_doc(adulib.llm.detokenize_text)

---

### detokenize_text

>      detokenize_text (tokens, llm_model)

In [ ]:
#|export
def detokenize_text(tokens, llm_model):
    if llm_model not in __model_tokenisers:
        __model_tokenisers[llm_model] = tiktoken.encoding_for_model(llm_model)
    tokeniser_enc = tiktoken.encoding_for_model(llm_model)
    return tokeniser_enc.decode(tokens)

In [ ]:
llm_model = 'gpt-4o'
detokenize_text(tokenize_text("hello world", llm_model), llm_model)

'hello world'

In [ ]:
show_doc(adulib.llm.get_token_count)

---

### get_token_count

>      get_token_count (text, llm_model)

In [ ]:
#|export
def get_token_count(text, llm_model):
    return len(tokenize_text(text, llm_model))

In [ ]:
get_token_count("hello world", "gpt-4o")

2

In [ ]:
show_doc(adulib.llm.async_prompt)

---

### async_prompt

>      async_prompt (model, prompt, context='You are a helpful assistant.',
>                    api_key=None, response_format=None, use_cache=True,
>                    cache_dir=None, include_model_in_cache_key=False,
>                    cache_key_prepend='')

In [ ]:
#|export
__client = None
__model_rate_limiters = {}
__llm_cache = None

async def async_prompt(model,
                           prompt,
                           context="You are a helpful assistant.",
                           api_key=None,
                           response_format=None,
                           use_cache=True,
                           cache_dir=None,
                           include_model_in_cache_key=False,
                           cache_key_prepend=''):
    global __client, __llm_cache
    if __client is None:
        if api_key is None: api_key = os.environ.get("PROJ_OPENAI_API_KEY")
        __client = AsyncOpenAI(api_key=api_key)
        
    if model not in __model_rate_limiters:
        __model_rate_limiters[model] = Limiter(model_rate_limits[model]/60)
        
    response_schema = str(response_format.model_json_schema()) if response_format else ""
        
    if __llm_cache is None:
        if cache_dir is None: cache_dir = tempfile.mkdtemp()
        abs_cache_path = Path(cache_dir).resolve().as_posix()
        __llm_cache = diskcache.Cache(abs_cache_path, eviction_policy="none", size_limit=2**40)
    
    _model_key = model if include_model_in_cache_key else '*'
    cache_key = f'{cache_key_prepend}:{_model_key}:{prompt}:{context}:{response_schema}'
    
    if use_cache and cache_key in __llm_cache:
        output =  __llm_cache[cache_key]
    else:
        await __model_rate_limiters[model].wait()
        if not response_schema:
            chat_completion = await __client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": context
                    },
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model=model,
            )
        else:
            chat_completion = await __client.beta.chat.completions.parse(
                messages=[
                    {
                        "role": "system",
                        "content": context
                    },
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model=model,
                response_format=response_format
            )
            
        output = chat_completion.choices[0].message.content
        __llm_cache[cache_key] = output
    
    if response_format:
        return response_format(**json.loads(output))
    else:
        return output

In [ ]:
await async_prompt(
    model='gpt-4o',
    context='You are a helpful assistant.',
    prompt='Hello, how are you?',
    cache_dir='./tmp/llm_cache'
)

"Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

In [ ]:
from pydantic import BaseModel
from pprint import pprint

In [ ]:
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str
    
res = await async_prompt(
    model='gpt-4o',
    context='You are a helpful math tutor. Guide the user through the solution step by step.',
    prompt='How can I solve 8x + 7 = -23?',
    response_format=MathReasoning,
    cache_dir='./tmp/llm_cache'
)

pprint(res.model_dump())

{'final_answer': 'x = -15/4 or x = -3.75',
 'steps': [{'explanation': 'The given equation is a linear equation in one '
                           'variable, where we need to isolate the variable x '
                           'to find its value. The equation is currently 8x + '
                           '7 = -23.',
            'output': 'Understand the objective: isolate x.'},
           {'explanation': 'Start by eliminating the constant term on the left '
                           'side of the equation, which is +7. We do this by '
                           'subtracting 7 from both sides of the equation.',
            'output': 'Subtract 7 from both sides: 8x + 7 - 7 = -23 - 7'},
           {'explanation': 'Simplify both sides of the equation by removing '
                           'the +7 and -7, since they cancel each other out on '
                           'the left side.',
            'output': 'Simplified to: 8x = -30'},
           {'explanation': 'Now, we have 8 times x e

In [ ]:
show_doc(adulib.llm.prompt)

---

### prompt

>      prompt (model, prompt, context='You are a helpful assistant.',
>              api_key=None, response_format=None, use_cache=True,
>              cache_dir=None, include_model_in_cache_key=False,
>              cache_key_prepend='')

In [ ]:
#|export
__client = None

def prompt(model,
                prompt,
                context="You are a helpful assistant.",
                api_key=None,
                response_format=None,
                use_cache=True,
                cache_dir=None,
                include_model_in_cache_key=False,
                cache_key_prepend=''):
    global __client, __llm_cache
    if __client is None:
        if api_key is None: api_key = os.environ.get("PROJ_OPENAI_API_KEY")
        __client = OpenAI(api_key=api_key)
        
    response_schema = str(response_format.model_json_schema()) if response_format else ""
        
    if __llm_cache is None:
        if cache_dir is None: cache_dir = tempfile.mkdtemp()
        abs_cache_path = Path(cache_dir).resolve().as_posix()
        __llm_cache = diskcache.Cache(abs_cache_path, eviction_policy="none", size_limit=2**40)
    
    _model_key = model if include_model_in_cache_key else '*'
    cache_key = f'{cache_key_prepend}:{_model_key}:{prompt}:{context}:{response_schema}'
    
    if use_cache and cache_key in __llm_cache:
        output =  __llm_cache[cache_key]
    else:
        if not response_schema:
            chat_completion = __client.chat.completions.create(  # Changed to synchronous call
                messages=[
                    {
                        "role": "system",
                        "content": context
                    },
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model=model,
            )
        else:
            chat_completion = __client.beta.chat.completions.parse(  # Changed to synchronous call
                messages=[
                    {
                        "role": "system",
                        "content": context
                    },
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model=model,
                response_format=response_format
            )
            
        output = chat_completion.choices[0].message.content
        __llm_cache[cache_key] = output
    
    if response_format:
        return response_format(**json.loads(output))
    else:
        return output

In [ ]:
prompt(
    model='gpt-4o',
    context='You are a helpful assistant.',
    prompt='Hello, how are you?',
    cache_dir='./tmp/llm_cache'
)

"Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"